In [1]:
import torch
from models.GooGleNet import GoogleNet
from models.CNNModels import train_model
%load_ext autoreload
%autoreload 2

Global seed set to 42


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = GoogleNet(c_in=3, num_classes=10, base_dim=64).to(device)
print(model)

In [70]:
x = torch.randn(1, 3, 200, 200).to(device)
output = model(x)

In [89]:
(output.argmax(dim=-1)==3).float().mean()

tensor(0., device='cuda:0')

In [79]:
import os
from torchvision.datasets import CIFAR10
DATASET_PATH = './data/CIFAR10'
if not os.path.isdir(DATASET_PATH):
    os.makedirs(DATASET_PATH, exist_ok=True)
train_data = CIFAR10(root=DATASET_PATH, train=True, download=True)

        
DATA_MEAN = (train_data.data / 255.0)

Files already downloaded and verified


In [19]:
train_model(
    model_name="GoogleNet",
    model_hparams={"num_classes": 10, 'c_in' : 3},
    optimizer_name="SGD",
    optimizer_hparams={"lr": 1e-3, "weight_decay": 1e-4},
    CHECKPOINT_PATH='./checkpoint'
)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [20]:
from models.CNNModels import CNNClassifier, CIFAR10DataModule

In [21]:
model = CNNClassifier(model_name='GoogleNet',
    model_hparams={"num_classes": 10, 'c_in' : 3},
    optimizer_name="SGD",
    optimizer_hparams={"lr": 1e-3, "weight_decay": 1e-4},)

In [22]:
import pytorch_lightning as pl
datamodule = CIFAR10DataModule()
trainer = pl.Trainer
trainer.fit(model, datamodule)

AttributeError: 'CNNClassifier' object has no attribute 'state'

In [14]:
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
import os

class LightningMNISTClassifier(pl.LightningModule):

  def __init__(self):
    super().__init__()

    # mnist images are (1, 28, 28) (channels, width, height) 
    self.layer_1 = torch.nn.Linear(28 * 28, 128)
    self.layer_2 = torch.nn.Linear(128, 256)
    self.layer_3 = torch.nn.Linear(256, 10)

  def forward(self, x):
      batch_size, channels, width, height = x.size()

      # (b, 1, 28, 28) -> (b, 1*28*28)
      x = x.view(batch_size, -1)

      # layer 1 (b, 1*28*28) -> (b, 128)
      x = self.layer_1(x)
      x = torch.relu(x)

      # layer 2 (b, 128) -> (b, 256)
      x = self.layer_2(x)
      x = torch.relu(x)

      # layer 3 (b, 256) -> (b, 10)
      x = self.layer_3(x)

      # probability distribution over labels
      x = torch.log_softmax(x, dim=1)

      return x

  def cross_entropy_loss(self, logits, labels):
    return F.nll_loss(logits, labels)

  def training_step(self, train_batch, batch_idx):
      x, y = train_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('train_loss', loss)
      return loss


  def validation_step(self, val_batch, batch_idx):
      x, y = val_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('val_loss', loss)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer


class MNISTDataModule(pl.LightningDataModule):

  def setup(self, stage):
    # transforms for images
    transform=transforms.Compose([transforms.ToTensor(), 
                                  transforms.Normalize((0.1307,), (0.3081,))])
      
    # prepare transforms standard to MNIST
    self.mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
    self.mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)

  def train_dataloader(self):
    return DataLoader(self.mnist_train, batch_size=64)

  def val_dataloader(self):
    return DataLoader(self.mnist_test, batch_size=64)

data_module = MNISTDataModule()

# train
model = LightningMNISTClassifier()
trainer = pl.Trainer()

trainer.fit(model, data_module)

Global seed set to 42
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\Users\spruce\anaconda3\envs\light\lib\site-packages\pytorch_lightning\trainer\trainer.py:1303: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(


100%|█████████▉| 9871360/9912422 [00:04<00:00, 2474292.25it/s]

KeyboardInterrupt: 

100%|█████████▉| 9887744/9912422 [00:20<00:00, 2474292.25it/s]